In [ ]:
import osmnx as ox
from shapely.geometry import box
from ipyleaflet import Map, basemaps, DrawControl, LayersControl, GeoData
from ipywidgets import Button, Output, VBox
import pandas as pd
from ipywidgets.embed import embed_minimal_html

# Initialize the map and output
output = Output()
m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=(29.3117, 47.4818), zoom=10)
m.add_control(DrawControl(rectangle={"shapeOptions": {"color": "#0000FF"}}))
m.add_control(LayersControl())

# Global variable to hold roundabout data
roundabout_data = pd.DataFrame()

# Functions for roundabout fetching, display, and exporting
def fetch_and_display_roundabouts(north, south, east, west):
    global roundabout_data
    polygon = box(west, south, east, north)
    tags = {'junction': 'roundabout'}
    roundabouts = ox.features_from_polygon(polygon, tags=tags)
    if not roundabouts.empty:
        roundabout_data = roundabouts[['geometry']].copy()
        roundabout_data['latitude'] = roundabout_data.geometry.centroid.y
        roundabout_data['longitude'] = roundabout_data.geometry.centroid.x
        geo_data = GeoData(
            geo_dataframe=roundabouts,
            style={'color': 'red', 'radius': 8, 'fillColor': 'red', 'opacity': 0.5, 'weight': 1.9, 'dashArray': '2', 'fillOpacity': 0.7},
            hover_style={'fillColor': 'red', 'fillOpacity': 0.2},
            name='Roundabouts'
        )
        m.add_layer(geo_data)
        with output:
            print(f"Found {len(roundabouts)} roundabouts.")
    else:
        with output:
            print("No roundabouts found in the selected area.")

def handle_draw(target, action, geo_json):
    if action == 'created':
        coords = geo_json['geometry']['coordinates'][0]
        north = max(coords, key=lambda x: x[1])[1]
        south = min(coords, key=lambda x: x[1])[1]
        east = max(coords, key=lambda x: x[0])[0]
        west = min(coords, key=lambda x: x[0])[0]
        fetch_and_display_roundabouts(north, south, east, west)

def export_excel(_):
    if not roundabout_data.empty:
        roundabout_data.to_excel('roundabouts.xlsx', index=False)
        with output:
            print("Roundabout data exported to 'roundabouts.xlsx'.")
    else:
        with output:
            print("No roundabout data to export.")

def export_map_image(_):
    with open("map_snapshot.html", "w") as f:
        embed_minimal_html("map_snapshot.html", views=[m])
    with output:
        print("Map snapshot saved as 'map_snapshot.html'. Open it in a browser and use a screenshot tool.")


In [ ]:
# Attach event handlers and display controls
draw_control = m.controls[0]
draw_control.on_draw(handle_draw)

excel_button = Button(description="Export Excel", button_style="success")
excel_button.on_click(export_excel)

image_button = Button(description="Export Map Image", button_style="info")
image_button.on_click(export_map_image)

display(VBox([m, excel_button, image_button, output]))
